# Prompt-to-Product: An AI Concept for Fashion Campaigns

## 🧠 Objective

This notebook presents a proof-of-concept tool demonstrating how generative AI can assist fashion and retail brands with early-stage marketing ideation. It focuses on the use of large language models (LLMs) and image generation to rapidly prototype campaign visuals and copy (i.e., short marketing text such as taglines).

The goal is to explore how structured inputs — like product name, aesthetic, and tone — can drive the automated creation of:

* Visual mockups aligned with brand style
* Campaign copy that resonates with a target audience (e.g., Gen Z)

This prototype simulates how creative and marketing teams might interact with lightweight AI tooling to speed up ideation, explore new directions, and align outputs with brand identity.

> **Note:** *This project is not affiliated with any specific brand. It is designed to be generalizable to modern, trend-oriented fashion marketing contexts.*

> **Note:** *This demo requires access to the OpenAI API and is not configured for public use due to API cost and access limitations.*


---



## 1. 📦 Setup and Imports



In [ ]:
# libraries
from openai import OpenAI
import random
from google.colab import userdata
import getpass
from IPython.display import Image, display
import requests
from PIL import Image as PILImage
from io import BytesIO

# Set OpenAI key - this key must be hidden for security purposes
api_key = userdata.get('OPENAI_API_KEY') # gets from google colab secrets
client = OpenAI(api_key=api_key) # sets key for API

## 2. 🧱 Prompt Builder

This section generates image prompts based on product name, style, and aesthetic.

Controlling prompts at this stage serves several key purposes:

* **Brand alignment**: Structured prompts ensure outputs stay on-message and visually consistent with the brand’s tone (e.g., Gen Z pastel vs vintage grunge).
* **Content safety**: By avoiding open-ended inputs, we reduce the risk of violating API usage policies or generating inappropriate content.
* **Prompt quality**: Good image generation depends on carefully crafted language. A structured builder reduces variability and increases the likelihood of usable outputs.
* **Scalability**: In a future tool, this type of prompt automation enables non-technical users (e.g., marketers) to generate assets safely without needing deep prompt engineering knowledge.

By templating prompts, we make AI generation safer, faster, and more effective for real-world creative teams.

**Note:** This is a rudimentary prompt control mechanism used only for proof-of-concept purposes. A production implementation would require more robust safeguards, including logging, throttling, and nuanced moderation handling.

In [ ]:
def build_prompt(product_name, aesthetic="Gen Z", style="editorial"):

    """
    Constructs a descriptive prompt for generative image models based on product name, aesthetic, and style.

    This function also checks the generated prompt against OpenAI's Moderation API to ensure it complies with
    content safety guidelines. If the prompt is flagged, it will not be returned.

    Parameters:
        product_name (str): The name or description of the product (e.g., "spring babydoll dress").
        aesthetic (str): The target visual or cultural aesthetic (e.g., "pastel Gen Z", "vintage 90s").
        style (str): The intended visual style of the image (e.g., "editorial", "flat lay").

    Returns:
        str or None: The validated prompt if safe, or None if the prompt was flagged by the moderation API.

    Notes:
        This is a basic implementation intended for proof-of-concept use only.
        Production versions should include more robust handling of flagged categories,
        user feedback, and logging.
    """

    prompt = f"{product_name}, {aesthetic} aesthetic, {style} style, fashion campaign mockup, 2025"

    # --- Moderation check using OpenAI v1 client ---
    moderation_response = client.moderations.create(input=prompt)
    moderation_result = moderation_response.results[0]

    # this warns user that they used inappropriate language for the tool
    if moderation_result.flagged:
        print("❌ Prompt flagged as unsafe.")
        print("⚠️ Categories:", [k for k, v in moderation_result.categories.dict().items() if v])
        return None

    print(f"✅ Prompt approved: '{prompt}'")
    return prompt


example_prompt = build_prompt("spring babydoll dress", aesthetic="pastel Gen Z", style="editorial")

`✅ Prompt approved: 'spring babydoll dress, pastel Gen Z aesthetic, editorial style, fashion campaign mockup, 2025'`

## 3. 🎨 Image Generation

This section uses the OpenAI API to generate campaign-style visuals from descriptive text prompts using the DALL·E model. These outputs can help marketing or creative teams explore seasonal aesthetics, alt colorways, or early campaign ideation.

📌 **Example Prompt Used**
"Urban Outfitters spring babydoll dress, Gen Z aesthetic, pastel colors, 2025 campaign mockup, editorial style"

### ⚠️ Common API Errors

When calling the image generation endpoint, you may encounter issues such as:

* **`401 Unauthorized`**: API key is missing or invalid.
* **`403 Forbidden`**: The model (e.g., DALL·E 3) is not available under your current access tier.
* **`429 Too Many Requests`**: You've hit a rate or usage limit.
* **`400 Bad Request`**: Invalid or unsupported parameters (e.g., using the wrong `size` with the model).
* **`500 Internal Server Error`**: Temporary issue on the API provider’s side.

For this proof of concept, basic error handling is included. A production version would expand on these cases, log, and guide users more clearly.

---

**Note**: The generated image includes multiple dresses. This is acceptable for the creative ideation context of campaign planning, where visual variety and aesthetic alignment are more important than precise product fidelity.


In [ ]:
def generate_dalle_image(prompt, size="1024x1024", model="dall-e-3"):

    """
    Generates and displays an image using OpenAI's image generation API (DALL·E) via the new client interface.

    Given a validated text prompt, this function sends a request to OpenAI's image generation endpoint,
    retrieves the resulting image, and displays it inline. It also returns the image URL for further use.

    Parameters:
        prompt (str): A descriptive, pre-moderated text prompt suitable for image generation.
        size (str): The desired image resolution.
                    - For 'dall-e-2': one of "256x256", "512x512", or "1024x1024".
                    - For 'dall-e-3': must be "1024x1024".
        model (str): The DALL·E model to use. Options include "dall-e-2" or "dall-e-3" (default).

    Returns:
        str or None: The URL of the generated image, or None if generation fails.

    Notes:
        - This function assumes the prompt has already passed moderation.
        - Requires an active and properly configured OpenAI client object.
        - For production use, consider adding rate limiting, larger sizes, quality selection,
          number of images for idea generation, exception handling for specific HTTP errors,
          and user feedback if generation fails due to model limitations or abuse of safeguards.
    """

    # dall-e 3's smallest size is 1024x1024
    if model == "dall-e-3" and size != "1024x1024":
      raise ValueError("dall-e-3 only supports size='1024x1024'")

    try:
        # generates the image
        response = client.images.generate(
            model=model,
            prompt=prompt,
            n=1,
            size=size,
            quality="standard"  # optional for dall-e-3
        )
        image_url = response.data[0].url

        # Fetch and display the image
        img_response = requests.get(image_url)
        img = PILImage.open(BytesIO(img_response.content))
        display(img)

        return image_url

    # if an error occurs return nothing and warn user
    except Exception as e:
        print("❌ Image generation failed:", str(e))
        return None


# Example use
generate_dalle_image(example_prompt)

Output hidden; open in https://colab.research.google.com to view.

![Daisy Dress](https://raw.githubusercontent.com/RobertLIII/FashionAI/master/daisy_dress.png)

## 4. ✍️ Campaign Copy Generator

This section uses the OpenAI Chat API to generate short, stylized campaign taglines (known as copy) based on the product name and desired tone. These taglines simulate early-stage marketing copy that could accompany a product in a seasonal lookbook, online catalog, or promotional ad.

The model is guided by a system prompt to behave like a brand copywriter, producing language aligned with a specific audience — such as Gen Z, minimalist, or vintage aesthetics.

🧠 **In Production Use:**
A robust version of this tool would likely generate **multiple tagline options** per product, allowing creative teams to iterate on language, compare tonal directions, and select or refine the best fit. Additionally, production systems could incorporate structured product metadata or even image-based analysis to further align copy with campaign visuals.


In [ ]:
def generate_tagline(product_name, tone="Gen Z"):
    """
    Generates a short fashion campaign tagline using OpenAI's GPT model based on product name and tone.

    Parameters:
        product_name (str): The name or label of the product (e.g., "The Daisy Slip Dress").
        tone (str): The desired tone or audience alignment (e.g., "Gen Z", "minimalist", "boho").

    Returns:
        str: A generated marketing tagline.
    """

    # create the tagline
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # or "gpt-3.5-turbo" if cost/access is a concern - parameterize?

            # set the user role so the GPT knows how to behave
            messages=[
                {"role": "system", "content": "You are a copywriter for a trendy fashion brand."},
                {"role": "user", "content": f"Write a short, catchy marketing tagline for a product called '{product_name}' in a {tone} tone."}
            ],
            temperature=0.8  # controls creativity
        )
        return response.choices[0].message.content.strip()

    # prints an error and returns nonthing
    except Exception as e:
        print("❌ Failed to generate tagline:", str(e))
        return None

generate_tagline("The Daisy Slip Dress", tone="Gen Z pastel")

Example Tagline: **"Embrace your Blossom Chic with The Daisy Slip Dress!"**

## 5. 📈 Business Case

### How This Supports the Brand

* Accelerates creative ideation
* Generates aesthetic-consistent variants
* Assists human creatives, doesn’t replace them
* Aligns with Gen Z engagement strategies
* Bridges creative and technical teams

### Future Steps

* **Deploy as a lightweight app**: An internal tool could allow marketers to generate and browse taglines or visuals interactively.
* **Integrate with product data**: Pull product names, categories, and metadata directly to automate prompt and copy generation at scale.
* **Generate multiple tagline options**: In production, offering several outputs per product would allow creative teams to iterate, compare, and refine selections.
* **Fine-tune models for brand voice**: Training or tuning could better match tone, phrasing, and formatting to the brand’s established copy style.
* **Expand to multimodal input**: Future versions may incorporate AI vision to write taglines directly from product imagery.


## 🔭 Production Considerations

While this notebook demonstrates a technical foundation for AI-assisted campaign ideation, a production version would require careful attention to additional factors beyond functionality. These include:

* **Robust content moderation** to prevent unsafe or inappropriate outputs
* **Throttling and cost control** to manage API usage at scale
* **User-facing error handling** with clear feedback loops
* **Brand tone alignment**, possibly via fine-tuned LLMs or curated prompt libraries
* **Creative review workflows**, where AI outputs are treated as drafts, not final assets


---


![Gen Z Diversity Representation](https://d27fp5ulgfd7w2.cloudfront.net/wp-content/uploads/2021/12/06165501/Diversity-Gen-Z-1-768x512.jpg)

*Image Source: [Global Marketing Alliance](https://www.the-gma.com/gen-z-and-diversity-how-the-next-generation-is-defining-inclusivity)*  
*Used here for educational and illustrative purposes.*


## 🌍 Ethical Design & Representation

In a production environment, it's crucial to ensure that AI-generated content reflects and respects the diversity of its audience. For brands targeting Gen Z, this means:

* **Inclusive Imagery**: Ensuring that generated visuals represent a range of genders, ethnicities, body types, and abilities.
* **Respectful Language**: Crafting copy that is free from stereotypes and resonates with diverse cultural backgrounds.
* **Bias Mitigation**: Implementing checks to prevent the reinforcement of societal biases in AI outputs.

> "81% of Gen Zers say multicultural and diverse communities greatly shape their brand preferences."
> — [Direct Digital Holdings Research](https://ir.directdigitalholdings.com/news-events/press-releases/detail/94/81-of-gen-zers-say-multiculturaldiverse-communities-greatly-shape-brand-preferences-according-to-direct-digital-holdings-research)

> "Gen Z consumers increasingly expect brands to act with authenticity, commit to diversity, and improve their eco-credentials."
> — [Mintel](https://www.mintel.com/insights/consumer-research/the-future-of-consumer-behaviour-in-the-age-of-gen-z/)

> "Creating user-centric AI product design, building a diverse team of reviewers and decision-makers, and rigorously auditing data sets are key factors."
> — [Entrepreneur](https://www.entrepreneur.com/growing-a-business/follow-these-5-principles-to-make-ai-more-inclusive-for-all/471915)

Incorporating these principles not only aligns with ethical standards but also enhances brand authenticity and connection with Gen Z audiences.
